<a href="https://colab.research.google.com/github/Blazkowiz47/object_detection_projects/blob/master/Casting_Faster_Rcnn_mmdetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %cd /content/drive/MyDrive/MMDetection Env/mmdetection
# MM Detection is in drive. just go there and install requirements.
# !git clone https://github.com/open-mmlab/mmdetection.git
# %cd mmdetection


In [ ]:
########################
# Run from here
%cd "/content/drive/MyDrive/MMDetection_Env/mmdetection"
!pip install mmcv-full
!pip install -r requirements.txt
#########################
# %cd mmdetection
# !mkdir checkpoints
# !wget -c https://openmmlab.oss-cn-hangzhou.aliyuncs.com/mmdetection/v2.0/vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-6879c318.pth\
#       -O checkpoints/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-6879c318.pth

In [ ]:
# %ls "/content/drive/MyDrive/MMDetection Env/mmdetection/checkpoints"

cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth
mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth
vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-6879c318.pth


In [2]:
from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, init_detector, inference_detector
import torch 

In [3]:
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
print(cfg)
cfg.dataset_type = 'CocoDataset'
cfg.data.classes = ("defect", "flaw",)
cfg.model.roi_head.bbox_head.num_classes = 2
# number of classes
# cfg.model.roi_head.bbox_head.num_classes = 14
# cfg.model.roi_head.mask_head.num_classes = 14

print(cfg)


Config (path: ./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py): {'model': {'type': 'FasterRCNN', 'backbone': {'type': 'ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': True, 'style': 'pytorch', 'init_cfg': {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}}, 'neck': {'type': 'FPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0]}, 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 1.0}}, 'roi_head': {'type': 'StandardRoIHead', 'bbox_roi_extractor': {'typ

In [ ]:
# !kaggle datasets download -d sreevishnudamodaran/vinbigdata-coco-dataset-with-wbf-3x-downscaled

In [ ]:
# ! unzip /root/vinbigdata-coco-dataset-with-wbf-3x-downscaled.zip -d vinbigdata-coco-dataset-with-wbf-3x-downscaled

In [4]:
PREFIX = '/content/drive/MyDrive/casting/'
cfg.data.train.img_prefix = PREFIX + 'train/'
cfg.data.train.ann_file = PREFIX + 'train/_annotations.coco.json'
cfg.data.train.classes = cfg.data.classes
cfg.data.train.type = 'CocoDataset'
cfg.data.val.img_prefix = PREFIX + 'val/'
cfg.data.val.classes = cfg.data.classes
cfg.data.val.ann_file = PREFIX + 'val/_annotations.coco.json'
cfg.data.val.type = 'CocoDataset'
cfg.data.test.img_prefix = PREFIX + 'test/'
cfg.data.test.classes = cfg.data.classes
cfg.data.test.ann_file = PREFIX + 'test/_annotations.coco.json'
cfg.data.test.type = 'CocoDataset'

In [5]:
cfg.log_config.interval = 1080
cfg.optimizer.lr = 0.02/10
cfg.lr_config.warmup = None
cfg.evaluation.metric = 'bbox'

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.load_from = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
cfg.work_dir = "../faster_rcnn"
cfg.runner.max_epochs = 2
cfg.total_epochs = 2

In [6]:
model = build_detector(cfg.model)
datasets = [build_dataset(cfg.data.train)]

/content/drive/MyDrive/MMDetection_Env/mmdetection/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '


loading annotations into memory...
Done (t=40.34s)
creating index...
index created!


In [ ]:
# !mkdir checkpoints
# !wget -c https://openmmlab.oss-cn-hangzhou.aliyuncs.com/mmdetection/v2.0/vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-6879c318.pth\
#       -O checkpoints/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-6879c318.pth

In [7]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True, )

loading annotations into memory...


2021-08-13 02:06:19,959 - mmdet - INFO - load checkpoint from checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
2021-08-13 02:06:19,967 - mmdet - INFO - Use load_from_local loader


Done (t=8.19s)
creating index...
index created!


2021-08-13 02:06:22,752 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([3, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([3]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([8, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([8]).
2021-08-13 02:06:22,758 - mmdet - INFO - Start running, host: root@5e905be3e1c1, work_dir: /content/drive/MyDrive/MMDetection_Env/faster_rcnn
2021-08-13 02:06:22,761 - mmdet - INFO - Hooks will be executed in the following order:
before_run

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1440/1440, 6.0 task/s, elapsed: 238s, ETA:     0s

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1440/1440, 10.0 task/s, elapsed: 145s, ETA:     0s

2021-08-13 03:00:30,784 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.47s).
Accumulating evaluation results...


2021-08-13 03:00:36,853 - mmdet - INFO - Epoch(val) [2][1440]	bbox_mAP: 0.8810, bbox_mAP_50: 0.9900, bbox_mAP_75: 0.9590, bbox_mAP_s: 0.4950, bbox_mAP_m: 0.8520, bbox_mAP_l: 0.9550, bbox_mAP_copypaste: 0.881 0.990 0.959 0.495 0.852 0.955


DONE (t=0.50s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.881
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.990
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.959
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.495
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.852
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.955
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.903
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.903
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.903
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.570
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.874
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.972


In [8]:
print(cfg.keys())
print(cfg.model.keys())
%cd
cfg.dump("/content/drive/MyDrive/MMDetection_Env/config.py")

dict_keys(['model', 'dataset_type', 'data_root', 'img_norm_cfg', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'log_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow', 'seed', 'gpu_ids', 'work_dir', 'total_epochs'])
dict_keys(['type', 'backbone', 'neck', 'rpn_head', 'roi_head', 'train_cfg', 'test_cfg'])
/root


In [9]:
!pip install mmdet
# !conda install pytorch==1.3.0 cudatoolkit=10.0 torchvision -c pytorch

# %cd "/content/drive/MyDrive/MMDetection_Env/mmdetection"
# MODEL = '../config.py'
# !python tools/train.py \
#   $MODEL \
#   --cfg-options dataset_type='CocoDataset' \
#   work_dir="../faster_rcnn"

     |████████████████████████████████| 1.1 MB 8.3 MB/s 


In [10]:
%cd "/content/drive/MyDrive/MMDetection_Env/mmdetection"
MODEL = '../config.py'
CHECKPOINT = '../faster_rcnn/latest.pth'
!python tools/test.py \
  $MODEL \
  $CHECKPOINT \
  --out results.pkl \
  --cfg-options dataset_type='CocoDataset' \
  work_dir="../faster_rcnn"


/content/drive/MyDrive/MMDetection_Env/mmdetection
loading annotations into memory...
Done (t=5.25s)
creating index...
index created!
/usr/local/lib/python3.7/dist-packages/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '
Use load_from_local loader
[                                                  ] 0/720, elapsed: 0s, ETA:/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/mmdet/core/anchor/anchor_generator.py:323: UserWarning: ``grid_anchors`` w

In [ ]:
# model = torch.load("/content/drive/MyDrive/MMDetection_Env/vinbig/epoch_9.pth", map_location=torch.device('cpu'))

In [11]:
# Classify the test images into good and bad
config = '../config.py'
result = '/content/drive/MyDrive/MMDetection_Env/mmdetection/results.pkl'
!python tools/analysis_tools/analyze_results.py \
      $config \
      $result \
      "../faster_rcnn/results" \


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
[>>] 720/720, 2.5 task/s, elapsed: 287s, ETA:     0s

In [20]:
log = "../faster_rcnn/20210731_111849.log.json"
!python tools/analysis_tools/analyze_logs.py plot_curve $log --keys acc --legend accuracy --out /content/drive/MyDrive/MMDetection_Env/faster_rcnn/results/20210731_111849_logs_acc_lr_0.002.pdf

plot curve of ../faster_rcnn/20210731_111849.log.json, metric is acc
save curve to: /content/drive/MyDrive/MMDetection_Env/faster_rcnn/results/20210731_111849_logs_acc_lr_0.002.pdf


In [18]:
log = "../faster_rcnn/None.log.modified.json"
!python tools/analysis_tools/analyze_logs.py plot_curve $log --keys acc --legend accuracy   --out /content/drive/MyDrive/MMDetection_Env/faster_rcnn/results/None_logs_acc_loss_lr_0.002.pdf

plot curve of ../faster_rcnn/None.log.modified.json, metric is acc
save curve to: /content/drive/MyDrive/MMDetection_Env/faster_rcnn/results/None_logs_acc_loss_lr_0.002.pdf


In [ ]:
import pickle
with open("/content/drive/MyDrive/MMDetection_Env/vinbig/results.pkl", 'wb') as f:
    pickle.dump(model,f)

In [ ]:
%cd "./"

/content/mmdetection


In [ ]:
%ls


checkpoints/  docs_zh-CN/      pytest.ini        resources/   tools/
configs/      LICENSE          README.md         results.pkl
demo/         MANIFEST.in      README_zh-CN.md   setup.cfg
docker/       mmdet/           requirements/     setup.py*
docs/         model-index.yml  requirements.txt  tests/


In [ ]:
{
  "model":{
    "type":"FasterRCNN",
    "backbone":{
      "type":"ResNet",
      "depth":50,
      "num_stages":4,
      "out_indices":(0,
      1,
      2,
      3),
      "frozen_stages":1,
      "norm_cfg":{
        "type":"BN",
        "requires_grad":true
      },
      "norm_eval":true,
      "style":"pytorch",
      "init_cfg":{
        "type":"Pretrained",
        "checkpoint":"torchvision://resnet50"
      }
    },
    "neck":{
      "type":"FPN",
      "in_channels":[
        256,
        512,
        1024,
        2048
      ],
      "out_channels":256,
      "num_outs":5
    },
    "rpn_head":{
      "type":"RPNHead",
      "in_channels":256,
      "feat_channels":256,
      "anchor_generator":{
        "type":"AnchorGenerator",
        "scales":[
          8
        ],
        "ratios":[
          0.5,
          1.0,
          2.0
        ],
        "strides":[
          4,
          8,
          16,
          32,
          64
        ]
      },
      "bbox_coder":{
        "type":"DeltaXYWHBBoxCoder",
        "target_means":[
          0.0,
          0.0,
          0.0,
          0.0
        ],
        "target_stds":[
          1.0,
          1.0,
          1.0,
          1.0
        ]
      },
      "loss_cls":{
        "type":"CrossEntropyLoss",
        "use_sigmoid":true,
        "loss_weight":1.0
      },
      "loss_bbox":{
        "type":"L1Loss",
        "loss_weight":1.0
      }
    },
    "roi_head":{
      "type":"StandardRoIHead",
      "bbox_roi_extractor":{
        "type":"SingleRoIExtractor",
        "roi_layer":{
          "type":"RoIAlign",
          "output_size":7,
          "sampling_ratio":0
        },
        "out_channels":256,
        "featmap_strides":[
          4,
          8,
          16,
          32
        ]
      },
      "bbox_head":{
        "type":"Shared2FCBBoxHead",
        "in_channels":256,
        "fc_out_channels":1024,
        "roi_feat_size":7,
        "num_classes":80,
        "bbox_coder":{
          "type":"DeltaXYWHBBoxCoder",
          "target_means":[
            0.0,
            0.0,
            0.0,
            0.0
          ],
          "target_stds":[
            0.1,
            0.1,
            0.2,
            0.2
          ]
        },
        "reg_class_agnostic":false,
        "loss_cls":{
          "type":"CrossEntropyLoss",
          "use_sigmoid":false,
          "loss_weight":1.0
        },
        "loss_bbox":{
          "type":"L1Loss",
          "loss_weight":1.0
        }
      }
    },
    "train_cfg":{
      "rpn":{
        "assigner":{
          "type":"MaxIoUAssigner",
          "pos_iou_thr":0.7,
          "neg_iou_thr":0.3,
          "min_pos_iou":0.3,
          "match_low_quality":true,
          "ignore_iof_thr":-1
        },
        "sampler":{
          "type":"RandomSampler",
          "num":256,
          "pos_fraction":0.5,
          "neg_pos_ub":-1,
          "add_gt_as_proposals":false
        },
        "allowed_border":-1,
        "pos_weight":-1,
        "debug":false
      },
      "rpn_proposal":{
        "nms_pre":2000,
        "max_per_img":1000,
        "nms":{
          "type":"nms",
          "iou_threshold":0.7
        },
        "min_bbox_size":0
      },
      "rcnn":{
        "assigner":{
          "type":"MaxIoUAssigner",
          "pos_iou_thr":0.5,
          "neg_iou_thr":0.5,
          "min_pos_iou":0.5,
          "match_low_quality":false,
          "ignore_iof_thr":-1
        },
        "sampler":{
          "type":"RandomSampler",
          "num":512,
          "pos_fraction":0.25,
          "neg_pos_ub":-1,
          "add_gt_as_proposals":true
        },
        "pos_weight":-1,
        "debug":false
      }
    },
    "test_cfg":{
      "rpn":{
        "nms_pre":1000,
        "max_per_img":1000,
        "nms":{
          "type":"nms",
          "iou_threshold":0.7
        },
        "min_bbox_size":0
      },
      "rcnn":{
        "score_thr":0.05,
        "nms":{
          "type":"nms",
          "iou_threshold":0.5
        },
        "max_per_img":100
      }
    }
  },
  "dataset_type":"CocoDataset",
  "data_root":"data/coco/",
  "img_norm_cfg":{
    "mean":[
      123.675,
      116.28,
      103.53
    ],
    "std":[
      58.395,
      57.12,
      57.375
    ],
    "to_rgb":true
  },
  "train_pipeline":[
    {
      "type":"LoadImageFromFile"
    },
    {
      "type":"LoadAnnotations",
      "with_bbox":true
    },
    {
      "type":"Resize",
      "img_scale":(1333,
      800),
      "keep_ratio":true
    },
    {
      "type":"RandomFlip",
      "flip_ratio":0.5
    },
    {
      "type":"Normalize",
      "mean":[
        123.675,
        116.28,
        103.53
      ],
      "std":[
        58.395,
        57.12,
        57.375
      ],
      "to_rgb":true
    },
    {
      "type":"Pad",
      "size_divisor":32
    },
    {
      "type":"DefaultFormatBundle"
    },
    {
      "type":"Collect",
      "keys":[
        "img",
        "gt_bboxes",
        "gt_labels"
      ]
    }
  ],
  "test_pipeline":[
    {
      "type":"LoadImageFromFile"
    },
    {
      "type":"MultiScaleFlipAug",
      "img_scale":(1333,
      800),
      "flip":false,
      "transforms":[
        {
          "type":"Resize",
          "keep_ratio":true
        },
        {
          "type":"RandomFlip"
        },
        {
          "type":"Normalize",
          "mean":[
            123.675,
            116.28,
            103.53
          ],
          "std":[
            58.395,
            57.12,
            57.375
          ],
          "to_rgb":true
        },
        {
          "type":"Pad",
          "size_divisor":32
        },
        {
          "type":"ImageToTensor",
          "keys":[
            "img"
          ]
        },
        {
          "type":"Collect",
          "keys":[
            "img"
          ]
        }
      ]
    }
  ],
  "data":{
    "samples_per_gpu":2,
    "workers_per_gpu":2,
    "train":{
      "type":"CocoDataset",
      "ann_file":"data/coco/annotations/instances_train2017.json",
      "img_prefix":"data/coco/train2017/",
      "pipeline":[
        {
          "type":"LoadImageFromFile"
        },
        {
          "type":"LoadAnnotations",
          "with_bbox":true
        },
        {
          "type":"Resize",
          "img_scale":(1333,
          800),
          "keep_ratio":true
        },
        {
          "type":"RandomFlip",
          "flip_ratio":0.5
        },
        {
          "type":"Normalize",
          "mean":[
            123.675,
            116.28,
            103.53
          ],
          "std":[
            58.395,
            57.12,
            57.375
          ],
          "to_rgb":true
        },
        {
          "type":"Pad",
          "size_divisor":32
        },
        {
          "type":"DefaultFormatBundle"
        },
        {
          "type":"Collect",
          "keys":[
            "img",
            "gt_bboxes",
            "gt_labels"
          ]
        }
      ]
    },
    "val":{
      "type":"CocoDataset",
      "ann_file":"data/coco/annotations/instances_val2017.json",
      "img_prefix":"data/coco/val2017/",
      "pipeline":[
        {
          "type":"LoadImageFromFile"
        },
        {
          "type":"MultiScaleFlipAug",
          "img_scale":(1333,
          800),
          "flip":false,
          "transforms":[
            {
              "type":"Resize",
              "keep_ratio":true
            },
            {
              "type":"RandomFlip"
            },
            {
              "type":"Normalize",
              "mean":[
                123.675,
                116.28,
                103.53
              ],
              "std":[
                58.395,
                57.12,
                57.375
              ],
              "to_rgb":true
            },
            {
              "type":"Pad",
              "size_divisor":32
            },
            {
              "type":"ImageToTensor",
              "keys":[
                "img"
              ]
            },
            {
              "type":"Collect",
              "keys":[
                "img"
              ]
            }
          ]
        }
      ]
    },
    "test":{
      "type":"CocoDataset",
      "ann_file":"data/coco/annotations/instances_val2017.json",
      "img_prefix":"data/coco/val2017/",
      "pipeline":[
        {
          "type":"LoadImageFromFile"
        },
        {
          "type":"MultiScaleFlipAug",
          "img_scale":(1333,
          800),
          "flip":false,
          "transforms":[
            {
              "type":"Resize",
              "keep_ratio":true
            },
            {
              "type":"RandomFlip"
            },
            {
              "type":"Normalize",
              "mean":[
                123.675,
                116.28,
                103.53
              ],
              "std":[
                58.395,
                57.12,
                57.375
              ],
              "to_rgb":true
            },
            {
              "type":"Pad",
              "size_divisor":32
            },
            {
              "type":"ImageToTensor",
              "keys":[
                "img"
              ]
            },
            {
              "type":"Collect",
              "keys":[
                "img"
              ]
            }
          ]
        }
      ]
    }
  },
  "evaluation":{
    "interval":1,
    "metric":"bbox"
  },
  "optimizer":{
    "type":"SGD",
    "lr":0.02,
    "momentum":0.9,
    "weight_decay":0.0001
  },
  "optimizer_config":{
    "grad_clip":"None"
  },
  "lr_config":{
    "policy":"step",
    "warmup":"linear",
    "warmup_iters":500,
    "warmup_ratio":0.001,
    "step":[
      8,
      11
    ]
  },
  "runner":{
    "type":"EpochBasedRunner",
    "max_epochs":12
  },
  "checkpoint_config":{
    "interval":1
  },
  "log_config":{
    "interval":50,
    "hooks":[
      {
        "type":"TextLoggerHook"
      }
    ]
  },
  "custom_hooks":[
    {
      "type":"NumClassCheckHook"
    }
  ],
  "dist_params":{
    "backend":"nccl"
  },
  "log_level":"INFO",
  "load_from":"None",
  "resume_from":"None",
  "workflow":[
    "(""train",
    1)
  ]
}